In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *

In [2]:
#Start session Context com nome edc
spark = SparkSession.builder.appName("EDC").getOrCreate()

In [4]:
%%time
edc = (
    spark
    .read
    .format('csv')
    .options(header=True, sep=";" )
    .option("encoding", "iso-8859-1")
    .load("MICRODADOS_CADASTRO_CURSOS_2019.csv")
 )


Wall time: 468 ms


In [5]:
%%time
edc.count()

Wall time: 948 ms


246081

In [6]:
edc.printSchema()

root
 |-- NU_ANO_CENSO: string (nullable = true)
 |-- NO_REGIAO: string (nullable = true)
 |-- CO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CO_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- CO_MUNICIPIO: string (nullable = true)
 |-- IN_CAPITAL: string (nullable = true)
 |-- TP_ORGANIZACAO_ACADEMICA: string (nullable = true)
 |-- TP_CATEGORIA_ADMINISTRATIVA: string (nullable = true)
 |-- TP_REDE: string (nullable = true)
 |-- CO_IES: string (nullable = true)
 |-- NO_CINE_ROTULO: string (nullable = true)
 |-- CO_CINE_ROTULO: string (nullable = true)
 |-- CO_CINE_AREA_GERAL: string (nullable = true)
 |-- NO_CINE_AREA_GERAL: string (nullable = true)
 |-- CO_CINE_AREA_ESPECIFICA: string (nullable = true)
 |-- NO_CINE_AREA_ESPECIFICA: string (nullable = true)
 |-- CO_CINE_AREA_DETALHADA: string (nullable = true)
 |-- NO_CINE_AREA_DETALHADA: string (nullable = true)
 |-- TP_GRAU_ACADEMICO: st

In [7]:
%%time
consulta1 = (
    edc
    .filter(col('QT_ING_CORND')== 1)
    .select('QT_INSCRITO_TOTAL')
    .agg(f.sum(col('QT_INSCRITO_TOTAL')).alias('total'))
)
consulta1.show()

+---------+
|    total|
+---------+
|1273332.0|
+---------+

Wall time: 1.23 s


In [8]:
#Qual é o número de alunos do Sexo Feminino estão no colegio no estado de código 35?
consulta2 = (
    edc 
    .select('NO_MUNICIPIO','NO_UF','CO_UF','SG_UF','QT_MAT_FEM')
    .filter(col('CO_UF')==31)
    .agg( f.sum(col('QT_MAT_FEM')).alias('total'))
    )
consulta2.show(8)

+--------+
|   total|
+--------+
|497560.0|
+--------+



In [9]:
#Quantos alunos do sexo feminino no estado de código 
consulta2 = (
    edc 
    .select('NO_MUNICIPIO','NO_UF','CO_UF','SG_UF','QT_MAT_FEM')
    .filter(col('CO_UF')==43)
    .agg( f.sum(col('QT_MAT_FEM')).alias('total'))
    )
consulta2.show(8)

+--------+
|   total|
+--------+
|298986.0|
+--------+



In [10]:
%%time
#matriculas femininas por estado
consulta3 = (
    edc 
    .select('NO_UF','CO_UF','QT_MAT_FEM')
    .groupby('NO_UF','CO_UF')
    .agg( f.sum(col('QT_MAT_FEM')).alias('total'))
    .sort(col("Total").desc())
    )
consulta3.show(8)

+-----------------+-----+---------+
|            NO_UF|CO_UF|    total|
+-----------------+-----+---------+
|        São Paulo|   35|1145411.0|
|     Minas Gerais|   31| 497560.0|
|   Rio de Janeiro|   33| 408263.0|
|           Paraná|   41| 317539.0|
|Rio Grande do Sul|   43| 298986.0|
|            Bahia|   29| 274264.0|
|   Santa Catarina|   42| 223208.0|
|            Ceará|   23| 176528.0|
+-----------------+-----+---------+
only showing top 8 rows

Wall time: 2.44 s


In [11]:
%%time
# DEFICIENTES POR ESTADO
consulta4 = (
     edc 
    .select('NO_UF','CO_UF','QT_ALUNO_DEFICIENTE')
    .groupby('NO_UF','CO_UF')
    .agg( f.sum(col('QT_ALUNO_DEFICIENTE')).alias('total'))
    .sort(col("Total").desc())
    

)
consulta4.show()

+-------------------+-----+-------+
|              NO_UF|CO_UF|  total|
+-------------------+-----+-------+
|          São Paulo|   35|14770.0|
|       Minas Gerais|   31| 6227.0|
|             Paraná|   41| 4424.0|
|  Rio Grande do Sul|   43| 4237.0|
|     Rio de Janeiro|   33| 4152.0|
|     Santa Catarina|   42| 3660.0|
|            Paraíba|   25| 2923.0|
|              Bahia|   29| 2659.0|
|              Ceará|   23| 2586.0|
|              Goiás|   52| 2119.0|
|               Acre|   12| 1857.0|
|               Pará|   15| 1837.0|
|        Mato Grosso|   51| 1692.0|
|   Distrito Federal|   53| 1555.0|
|           Maranhão|   21| 1421.0|
|         Pernambuco|   26| 1413.0|
| Mato Grosso do Sul|   50| 1393.0|
|     Espírito Santo|   32| 1240.0|
|            Alagoas|   27| 1122.0|
|Rio Grande do Norte|   24| 1042.0|
+-------------------+-----+-------+
only showing top 20 rows

Wall time: 1.64 s


In [12]:
%%time
# PRETA POR ESTADO
consulta3 = (
    edc 
    .select('NO_UF','CO_UF','QT_ING_PRETA')
    .groupby('NO_UF','CO_UF')
    .agg( f.sum(col('QT_ING_PRETA')).alias('total'))
    .sort(col("Total").desc())
    )
consulta3.show(8)

+-----------------+-----+-------+
|            NO_UF|CO_UF|  total|
+-----------------+-----+-------+
|        São Paulo|   35|55350.0|
|            Bahia|   29|30793.0|
|   Rio de Janeiro|   33|30154.0|
|     Minas Gerais|   31|29318.0|
|Rio Grande do Sul|   43|10774.0|
|            Goiás|   52| 8057.0|
| Distrito Federal|   53| 7846.0|
|         Maranhão|   21| 7438.0|
+-----------------+-----+-------+
only showing top 8 rows

Wall time: 1.45 s


In [14]:
%%time
#Quantos casos foram reportados nesse dia?
consulta4= (
        edc
         .select('NO_CINE_ROTULO')
         .filter(col('NO_CINE_ROTULO').rlike ('.*dados*.'))

            )
consulta4.count()

Wall time: 896 ms


353